Datasets taken from kaggle links below:

https://www.kaggle.com/datasets/udaysankarmukherjee/furniture-image-dataset (For tv and table)

https://www.kaggle.com/datasets/arminajdehnia/antic-chairs (for chair)

https://www.kaggle.com/datasets/sunnykusawa/sofas (for sofa)

In [1]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

In [2]:
# ===========================
# 1. Load & Augment Dataset
# ===========================

batch_size = 32
img_height = 180
img_width = 180
validation_split = 0.2  # 20% of the data will be used for validation

# Data Augmentation for Training Data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=validation_split  # Spliting the data into training and validation sets
)

# Preprocessing for Validation Data (No Augmentation)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split  # Spliting the data into training and validation sets
)

# Train Data Generator
train_generator = train_datagen.flow_from_directory(
    'C:/student/AIML/PROJECT/PART B DATASETS',  # this is my directory thats containing all the images
    target_size=(img_height, img_width),  
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify that this is the training set
)

# Validation Data Generator
validation_generator = test_datagen.flow_from_directory(
    'C:/student/AIML/PROJECT/PART B DATASETS',  # Same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 7123 images belonging to 4 classes.
Found 1780 images belonging to 4 classes.


In [3]:
# Class labels
labels = train_generator.class_indices
labels = dict((v, k) for k, v in labels.items())
print("Class labels:", labels)

Class labels: {0: 'Chair_Images', 1: 'Sofa_Images', 2: 'TV_Images', 3: 'Table_Images'}


In [4]:
# ===========================
# 2. Build CNN Model (RESNET50)
# ===========================

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # for freezing pretrained layers

In [5]:
# Creating new model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

In [6]:
# Print model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 6, 6, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               9437312   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 33,025,540
Trainable params: 9,437,828
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [7]:
# ===========================
# 3. Compile & Train Model
# ===========================

# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [8]:
# Train model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    verbose=1,
)

Epoch 1/10
223/223 [==============================] - 158s 709ms/step - loss: 1.3731 - accuracy: 0.5856 - val_loss: 0.4778 - val_accuracy: 0.8225
Epoch 2/10
223/223 [==============================] - 157s 706ms/step - loss: 0.6196 - accuracy: 0.7182 - val_loss: 0.4745 - val_accuracy: 0.8219
Epoch 3/10
223/223 [==============================] - 157s 704ms/step - loss: 0.5668 - accuracy: 0.7429 - val_loss: 0.3254 - val_accuracy: 0.8871
Epoch 4/10
223/223 [==============================] - 157s 703ms/step - loss: 0.4981 - accuracy: 0.7644 - val_loss: 0.2317 - val_accuracy: 0.9404
Epoch 5/10
223/223 [==============================] - 158s 710ms/step - loss: 0.4606 - accuracy: 0.7814 - val_loss: 0.1935 - val_accuracy: 0.9511
Epoch 6/10
223/223 [==============================] - 158s 707ms/step - loss: 0.4188 - accuracy: 0.7984 - val_loss: 0.1619 - val_accuracy: 0.9551
Epoch 7/10
223/223 [==============================] - 158s 707ms/step - loss: 0.4287 - accuracy: 0.7915 - val_loss: 0.1396 -

In [9]:
# ===========================
# 4. Save Trained Model
# ===========================

model.save("furniture_model_RESNET.h5")
print("Saved model as furniture_model_RESNET.h5")

Saved model as furniture_model_RESNET.h5


In [10]:
# ===========================
# 5. Evaluate Model
# ===========================

scores = model.evaluate(validation_generator, verbose=1)
print(f"Validation Accuracy: {scores[1] * 100:.2f}%")


56/56 [==============================] - 30s 540ms/step - loss: 0.2963 - accuracy: 0.9545
Validation Accuracy: 95.45%
